In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor

from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

In [4]:
train = pd.read_csv('/kaggle/input/mh-work-hour-prediction/train.csv')
test = pd.read_csv('/kaggle/input/mh-work-hour-prediction/test.csv')
submission = pd.read_csv('/kaggle/input/mh-work-hour-prediction/submission.csv')

In [5]:
train_orig = train.copy()
test_orig = test.copy()
submission_orig = submission.copy()

In [6]:
train.shape, test.shape, submission.shape

In [7]:
train.index, test.index, submission.index

In [8]:
train.columns, test.columns, submission.columns

In [9]:
train.columns.difference(test.columns), train.columns.intersection(submission.columns)

In [10]:
#tgt_var = train.columns.difference(test.columns).values[1]
tgt_var = submission.columns
tgt_var

In [11]:
train.info()
train.head(1)

In [12]:
display(submission['hours-per-week'].nunique(), submission['hours-per-week'].unique()[0], submission['hours-per-week'].value_counts())

In [13]:
display(train.isna().sum(), test.isna().sum())

In [14]:
train.duplicated().sum(), test.duplicated().sum()

In [15]:
def combinfo(data) :
    combi = pd.DataFrame(data.nunique(),columns=['Nunique()'])
    combi2 = pd.DataFrame(data.isna().sum(),columns=['isna() count'])
    combi3 = pd.concat([combi, combi2],axis=1)

    import io
    buffer = io.StringIO()
    data.info(buf=buffer, memory_usage=False)
    s = buffer.getvalue()
    df = pd.DataFrame(s.split("\n"),columns=['info'])
    df_info = df.loc[5:len(data.columns)+5-1,'info'].str.split(n=4,expand=True).reset_index(drop=True)
    df_info.drop(columns=[0,3],axis=1,inplace=True)
    df_info.columns = ['Column','Non-Null count','Dtype']
    df_info.set_index('Column',inplace=True)

    combi5 = pd.concat([combi3, df_info],axis=1)
    return combi5

def null_percent(data) :
    print("for all features")
    [print(data[i].isna().sum(),"(",round((data[i].isnull().sum()/len(data[i]))*100,2),"%)-->",i) \
     for i in data.columns if data[i].isna().sum()>0]
    
def vc(data, max_count,sort_idx) :
    [print(data[i].value_counts().sort_index()) for i in data.columns if data[i].nunique() < max_count]

def corrmat(data, k=20): #number of variables for heatmap
    #Outcome correlation matrix
    ##ck = 20 #number of variables for heatmap
    corrmat = data.corr()
    display(corrmat)
    cols = corrmat.nlargest(k, tgt_var)[tgt_var].index
    display(cols)
    cm = np.corrcoef(data[cols].values.T)
    display(cm)
    sns.set(font_scale=1.25)
    plt.figure(figsize=(12,9))
    hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
    plt.show()

In [16]:
combinfo(train)

In [17]:
null_percent(train)

In [18]:
vc(train,25,True)

In [19]:
train

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn import set_config
set_config(print_changed_only=True)
df = pd.DataFrame({'feature_1': np.random.rand(20,),
                   'feature_2': np.random.choice(['male', 'female'], (20,)),
                   'feature_3': np.random.choice(['single','married'],(20,))})
target = pd.Series(np.random.choice(['yes', 'no'], (20,)))
print(df)
print(target)

model = Pipeline([('preprocess',
                   ColumnTransformer([('ohe',
                                       OneHotEncoder(handle_unknown='ignore'), ['feature_2','feature_3'])], #)),
                                       remainder='passthrough')),
                  ('lr', LogisticRegression())],verbose=True)
'''
model = Pipeline([('ctenc',ColumnTransformer([('rep',pd.columns.replace({'female':0,'male':1},inplace=True),[1])])),
                    #('enc',df['feature_2'].replace({'female':0,'male':1},inplace=True)),
                    ('lr', LogisticRegression())])
'''
print(df)
model.fit(df, target)

# let us introduce new categories in feature_2 in test data
test_df = pd.DataFrame({'feature_1': np.random.rand(20),
                        'feature_2': np.random.choice(['male', 'female', 'neutral', 'unknown'], (20,)),
                        'feature_3': np.random.choice(['single','married','divorced','widowed'],(20,))})
model.predict(test_df)

In [21]:
train

In [22]:
train["source"] = "train"
test["source"] = "test"
data = pd.concat([train,test])
data_orig = data.copy()
display(data.head(2))
data = data.reset_index().drop(columns="index")
print(train.shape[0]+test.shape[0], data.shape, data.index)

In [23]:
cat_col_names = train.select_dtypes(exclude = np.number).drop(columns='source').columns
display(cat_col_names)
num_col_names = train.select_dtypes(include = np.number).drop(columns=tgt_var).columns
display(num_col_names,train[num_col_names])
#ct = make_column_transformer((OneHotEncoder(), cat_col_names), remainder = 'passthrough')
#ct = ColumnTransformer([('ohe',OneHotEncoder(handle_unknown='ignore'), cat_col_names)]
#                  ,remainder='passthrough')
#ct_train = ct.fit_transform(train)
#print(type(ct_train))


In [50]:
preprocess_cat_data_columns = pd.get_dummies(data[cat_col_names],drop_first=True)
display(preprocess_cat_data_columns.head())
preprocess_cat_data_columns.rename(columns={'income_ >50K':'income_gt_50k'},inplace=True)
scaler = StandardScaler()
scaler.fit(data[num_col_names])
data[num_col_names] = scaler.transform(data[num_col_names])    
    
final_preprocessed = pd.concat([data[num_col_names],preprocess_cat_data_columns,data[tgt_var],data['source']], axis = 1)
display(final_preprocessed.head())

In [51]:
display(final_preprocessed["hours-per-week"])

In [52]:
plt.figure(figsize=(20,20))
#seaborn.heatmap(final_preprocessed.corr(), annot=True)

In [53]:
train = final_preprocessed[final_preprocessed['source']=='train']
test = final_preprocessed[final_preprocessed['source']=='test']
del train['source'], test['source']


In [54]:
train

In [55]:
X = train.drop(columns='hours-per-week')
y = train['hours-per-week']
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2)

In [56]:
def model_fit(model, train_X, test_X, train_y, test_y):
    model.fit(train_X, train_y)
    train_pred = model.predict(train_X)
    test_pred = model.predict(test_X)
    train_acc = mean_squared_error(train_y, train_pred,squared=False )
    test_acc = mean_squared_error(test_y, test_pred,squared=False )
    print(model)
    print("Train Accuracy = ", train_acc)
    print("Test Accuracy = ", test_acc)
    return train_acc, test_acc

In [57]:
run_model = [LinearRegression(), Lasso(), 
             Ridge(),ElasticNet(l1_ratio=0.5),
             KNeighborsRegressor(n_neighbors=5),SVR(),
            DecisionTreeRegressor(), RandomForestRegressor(bootstrap='False'), AdaBoostRegressor(),
            GradientBoostingRegressor(),XGBRegressor(),LGBMRegressor(),CatBoostRegressor()]

model_name = ['LinearRegression', 'Lasso', 'Ridge',
            'Elasticnet','KNeighborsRegressor','SVM',
            'DecisionTreeRegressor', 'RandomForestRegressor', 'AdaBoostRegressor',
            'GradientBoostingRegressor','XGBRegressor','LGBMRegressor','CatBoostRegressor']

In [60]:

train_acc = []
test_acc = []
for i in run_model:
    trainacc, testacc = model_fit(i, train_X, test_X, train_y, test_y)
    train_acc.append(trainacc)
    test_acc.append(testacc)

In [59]:
X.columns.T

In [67]:
cmp = pd.DataFrame([train_acc,test_acc],columns=model_name).T
cmp